In [1]:
%env MLFLOW_TRACKING_URI=http://127.0.0.1:5001

env: MLFLOW_TRACKING_URI=http://127.0.0.1:5001


In [2]:
import mlflow

import numpy as np
from sklearn import datasets, metrics
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split


def eval_metrics(pred, actual):
    rmse = np.sqrt(metrics.mean_squared_error(actual, pred))
    mae = metrics.mean_absolute_error(actual, pred)
    r2 = metrics.r2_score(actual, pred)
    return rmse, mae, r2

In [3]:
# Set th experiment name
mlflow.set_experiment("wine-quality")
# Enable auto-logging to MLflow
mlflow.sklearn.autolog()

In [4]:
# Load wine quality dataset
X, y = datasets.load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Start a run and train a model
with mlflow.start_run(run_name="default-params"):
    lr = ElasticNet()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)
    rmse, mae, r2 = eval_metrics(y_pred, y_test)
    mlflow.log_metrics(
        {
            "mean_squared_error_X_test": rmse,
            "mean_absolute_error_X_test": mae,
            "r2_score_X_test": r2,
        }
    )

🏃 View run default-params at: http://127.0.0.1:5001/#/experiments/642291859068461495/runs/1d4056adae574eefa949d64ecee27fdc
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/642291859068461495


In [5]:
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV

lr = ElasticNet()

# Define distribution to pick parameter values from
distributions = dict(
    alpha=uniform(loc=0, scale=10),  # sample alpha uniformly from [-5.0, 5.0]
    l1_ratio=uniform(),  # sample l1_ratio uniformlyfrom [0, 1.0]
)

# Initialize random search instance
clf = RandomizedSearchCV(
    estimator=lr,
    param_distributions=distributions,
    # Optimize for mean absolute error
    scoring="neg_mean_absolute_error",
    # Use 5-fold cross validation
    cv=5,
    # Try 100 samples. Note that MLflow only logs the top 5 runs.
    n_iter=100,
)

# Start a parent run
with mlflow.start_run(run_name="hyperparameter-tuning"):
    search = clf.fit(X_train, y_train)

    # Evaluate the best model on test dataset
    y_pred = clf.best_estimator_.predict(X_test)
    rmse, mae, r2 = eval_metrics(y_pred, y_test)
    mlflow.log_metrics(
        {
            "mean_squared_error_X_test": rmse,
            "mean_absolute_error_X_test": mae,
            "r2_score_X_test": r2,
        }
    )


2024/11/26 09:57:14 INFO mlflow.sklearn.utils: Logging the 5 best runs, 95 runs will be omitted.


🏃 View run ambitious-carp-274 at: http://127.0.0.1:5001/#/experiments/642291859068461495/runs/75f6bd22b3994942b85bd78db0088da7
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/642291859068461495
🏃 View run brawny-squirrel-278 at: http://127.0.0.1:5001/#/experiments/642291859068461495/runs/7642f28541324519b76c051b19db9f9e
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/642291859068461495
🏃 View run gregarious-skink-652 at: http://127.0.0.1:5001/#/experiments/642291859068461495/runs/509c097f7bcb49b286c4012ba0c1e8b9
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/642291859068461495
🏃 View run awesome-boar-720 at: http://127.0.0.1:5001/#/experiments/642291859068461495/runs/a336f6375ca846e38c02703036f9034d
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/642291859068461495
🏃 View run hyperparameter-tuning at: http://127.0.0.1:5001/#/experiments/642291859068461495/runs/5b987155f66343e495e0aa169e2c0b4d
🧪 View experiment at: http://127.0.0.1:5001/#/experim

🏃 View run mercurial-boar-183 at: http://127.0.0.1:5001/#/experiments/642291859068461495/runs/6a2f61cfcace4baf902e5a39f31f5a7b
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/642291859068461495
